# LeetCode 1765
![lc-1765](./assets/question.jpg)
![lc-1765](./assets/constraints.jpg)

> Observations:
> - For all given matrices, there is always AT LESAT ONE water cell
> - For isWater, 1 = water level and 0 = land cell (with to-be defined level)
> - Any adjacent cells must have an absolute height difference of at most 1
> - Note that matrix is of m * n, not n * n

![lc-1765-ex1](./assets/ex1.jpg)
![lc-1765-ex2](./assets/ex2.jpg)

> Notes:
> - All cells next to water cells are at an elevation level of one unit, while adjacent cells follow the rule of being at most one level of elevation higher
> - With the note above, this question suggest the use of a breadth first search (BFS) solution implementation where we first find all locations of sea-level cells (water cells)
> - Then, with inspiration from the A-star pathfinder algorithm, we move in the cardinal directions to set the sea level elevations on the matrix
> - But of course, we do not want to overwrite any of the cells we have already visited, so we need a list to keep track of all cell locations that have been visited

### Initial Algorithm
> - We need a list for storing visited locations 
> - We need a list to act as our queue for BFS to control when to set a particular level of elevation
> - We first traverse the matrix to gather all the locations of the sea-level elevations (1's on the isWater matrix)
>   - We add this location to visited and enqueue these locations to be set at a particular level of elevation
> - Then while the queue is not empty, we move through the queue and set the height whilst also enqueuing any possible neighboring cells
>   - We need a logical statement to decide if a particular cell should be enqueued in order to be later set with a sea-level elevation
>   - Conditions for cell to not be enqueued:
>       - if column is out of bounds -> col < 0 or col > num_of_columns
>       - if row is out of bounds -> row < 0 or row > num_of_rows
>       - if is already visited
> - The step above is repeated till the queue is empty

## Initial Implementation

In [1]:
class Solution:
    def highestPeak(self, isWater):
        visited = []
        elevation_queue = []
        elevation = 0

        def validateCell(row, col):
            if ((row < 0) or (row > len(isWater) - 1) or (col < 0) or (col > len(isWater[0]) - 1) or ([row, col] in visited)):
                return
            visited.append([row, col])
            elevation_queue.append([row, col])

        for i in range(len(isWater)):
            for j in range(len(isWater[0])):
                if (isWater[i][j]):
                    visited.append([i, j])
                    elevation_queue.append([i, j])

        while elevation_queue:
            for i in range(len(elevation_queue)):
                row, col = elevation_queue.pop(0)
                isWater[row][col] = elevation
                validateCell(row + 1, col)
                validateCell(row - 1, col)
                validateCell(row, col + 1)
                validateCell(row, col - 1)
            elevation += 1

        return isWater
                

In [3]:
sol = Solution()
print('Result Ex1:', sol.highestPeak([[0,1],[0,0]]))
print('Result Ex2:', sol.highestPeak([[0,0,1],[1,0,0],[0,0,0]]))

Result Ex1: [[1, 0], [2, 1]]
Result Ex2: [[1, 1, 0], [0, 1, 1], [1, 2, 2]]


> Additional Notes:
> - After submission to LeetCode, the current implementation runs into the issue of a Time Limit Exceeded issue, so we now must attempt at finding a solution to make the algorithm more efficient
> - Currently the algorithm attempts to remove all elements from the elevation_queue list; which is fine. However, we call a helper function to check if the cell is valid for determining its elevation
> - Naively, to solve the problem, a visited list was utilised which copmrimises the space complexity, since the grid isWater is only initially filled with 1's and 0's, then there may be an implementation such that we remove the need for a visited list; that or we create a new matrix of same dims as isWater to then return as the solution
> - Could we start by setting the water to level 0? The elevation is based on the already-set sea level elevation which can be passed as a member of the queue as well? 

## Final Implementation

In [23]:
class Solution:
    def highestPeak(self, isWater):
        m, n = len(isWater), len(isWater[0])
        elevation_map = [[-1 for _ in range(n)] for _ in range(m)]
        elevation_queue = []
        
        for i in range(m): 
            for j in range(n):
                if (isWater[i][j]):
                    elevation_map[i][j] = 0
                    elevation_queue.append([i, j, 0])
        
        while elevation_queue:
            row, col, elevation = elevation_queue.pop(0)
            for delta_row, delta_col in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                if ((0 <= row + delta_row < m) and (0 <= col + delta_col < n) and elevation_map[row + delta_row][col + delta_col] == -1):
                    elevation_map[row + delta_row][col + delta_col] = elevation + 1
                    elevation_queue.append([row + delta_row, col + delta_col, elevation + 1])

        return elevation_map 

In [24]:
sol = Solution()
print('Result Ex1:', sol.highestPeak([[0,1],[0,0]]))
print('Result Ex2:', sol.highestPeak([[0,0,1],[1,0,0],[0,0,0]]))

Result Ex1: [[1, 0], [2, 1]]
Result Ex2: [[1, 1, 0], [0, 1, 1], [1, 2, 2]]
